# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-080ba229ef-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series (Basketball and NBA seasons explorative search") 

Consider the following exploratory information need:

>  investigate the world of basketball. Compare french teams and US teams, country of citizenship of NBA players, and teams which won NBA seasons.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q41421`| Michael Jordan     | node |
| `wd:Q25369`| Kobe Bryant     | node |





Also consider

```
wd:Q41421 ?p ?obj .
```

is the BGP to retrieve all **properties of Michael Jordan**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for basketball player.

2. Identify the BGP for basketball.

3. Identify the BGP for basketball team.

4. Return all french basketball teams born before 1930 (the result set must be a list of triples basketball team IRI, label and year of inception).

5. Return all the NBA season (the result set must be a list of couples NBA season IRI, label)

6. Consider only the basketball teams which Michael Jordan and Kobe Bryant played. Return the number of NBA seasons won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team even if they never won a season).

7. Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, label and #players)

8. Identify the BGP for the NBA Defensive Player of the Year Award (see https://en.wikipedia.org/wiki/NBA_Defensive_Player_of_the_Year_Award)

9. Consider the players who won the NBA Defensive Player of the Year Award. Return for these players the number of NBA Defensive Player of the Year Award won in their career (the result set must be triples of player IRI, label and #victories)


## Task 1
Identify the BGP for basketball player.

In [3]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q41421 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1048'), ('name', 'NCL ID')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p'

In [6]:
# Get occupation of Michael Jordan
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q41421 wdt:P106 ?o .
   ?o sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q131524'), ('name', 'entrepreneur')]
[('o', 'http://www.wikidata.org/entity/Q18814623'), ('name', 'autobiographer')]
[('o', 'http://www.wikidata.org/entity/Q33999'), ('name', 'actor')]
[('o', 'http://www.wikidata.org/entity/Q3665646'), ('name', 'basketball player')]
[('o', 'http://www.wikidata.org/entity/Q10871364'), ('name', 'baseball player')]
[('o', 'http://www.wikidata.org/entity/Q17221'), ('name', 'spokesperson')]
6


Final query for this task

In [3]:
# Get occupation of Michael Jordan
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q41421 wdt:P106 ?o .
   ?o sc:name ?name.
   FILTER(?o = wd:Q3665646)
}
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3665646'), ('name', 'basketball player')]
1


## Task 2
Identify the BGP for basketball.

In [7]:
# Get all properties of 'Basketball player' (Q3665646)
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q3665646 ?p ?obj .
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.w

Final query for this task

In [8]:
# Get value for property 'sport'
queryString = """
SELECT DISTINCT ?sport ?name
WHERE {
   wd:Q3665646 wdt:P641 ?sport .
   ?sport sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
1


## Task 3
Identify the BGP for basketball team.

Final query for this task

In [9]:
# Get value for property 'part of'
queryString = """
SELECT DISTINCT ?partOf ?name
WHERE {
   wd:Q3665646 wdt:P361 ?partOf .
   ?partOf sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('partOf', 'http://www.wikidata.org/entity/Q13393265'), ('name', 'basketball team')]
1


## Task 4
Return all french basketball teams born before 1930 (the result set must be a list of triples basketball team IRI, label and year of inception).

In [10]:
# Get all properties of 'basketball team'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q13393265 ?p ?o .
   ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
8


In [12]:
# Get super class of 'basketball team'
queryString = """
SELECT DISTINCT ?super ?name
WHERE {
   wd:Q13393265 wdt:P279 ?super .
   ?super sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('super', 'http://www.wikidata.org/entity/Q847017'), ('name', 'sports club')]
[('super', 'http://www.wikidata.org/entity/Q12973014'), ('name', 'sports team')]
2


In [13]:
# Get all properties of 'sports club'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q847017 ?p ?o .
   ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidat

In [14]:
# Get all properties of 'sports team'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q12973014 ?p ?o .
   ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P3827'), ('name',

In [17]:
# Get 'commons category' of 'basketball team'
queryString = """
SELECT DISTINCT ?cc ?name
WHERE {
   wd:Q13393265 wdt:P373 ?cc .
   ?cc sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [19]:
# Get property 'sport' of 'basketball team'
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q13393265 wdt:P641 ?o .
   ?o sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
1


In [21]:
# Get properties of 'basketball'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q5372 ?p ?o .
   ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1807'), ('name', 'Great Aragonese En

In [3]:
# Get instances of 'basketball team'
queryString = """
SELECT DISTINCT ?bteam ?name
WHERE {
   ?bteam wdt:P31 wd:Q13393265 .
   ?bteam sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('bteam', 'http://www.wikidata.org/entity/Q8206968'), ('name', 'Hebraica y Macabi')]
[('bteam', 'http://www.wikidata.org/entity/Q5144784'), ('name', "Colgate Raiders men's basketball")]
[('bteam', 'http://www.wikidata.org/entity/Q7051115'), ('name', "Norfolk State Spartans men's basketball")]
[('bteam', 'http://www.wikidata.org/entity/Q7397248'), ('name', "Sacred Heart Pioneers men's basketball")]
[('bteam', 'http://www.wikidata.org/entity/Q15725839'), ('name', 'A.S. Kentavros Neas Kallikrateias')]
[('bteam', 'http://www.wikidata.org/entity/Q107876527'), ('name', "Iowa Western Reivers men's basketball")]
[('bteam', 'http://www.wikidata.org/entity/Q15927936'), ('name', 'Westchester Knicks')]
[('bteam', 'http://www.wikidata.org/entity/Q810271'), ('name', 'BC Zepter Vienna')]
[('bteam', 'http://www.wikidata.org/entity/Q100320348'), ('name', "East Central Tigers men's basketball")]
[('bteam', 'http://www.wikidata.org/entity/Q16201949'), ('name', "DePaul Blue Demons women's basketb

In [5]:
# Get properties of 'Alabama State Hornets men's basketball'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q4705333 ?p ?o.
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P6112'), ('name', 'category for members of a team')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P831'), ('name', 'parent club')]
[('p', 'http://www.wikidata.org/prop/direct/P856'), ('name', 'official website')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
[('p', 'http://www.wikidata.org/prop/direct/P8825'), ('name', 'Sports-Reference.com college basketball school ID')]
12


In [6]:
# Get country of 'Alabama State Hornets men's basketball'
queryString = """
SELECT DISTINCT ?country ?name
WHERE {
   wd:Q4705333 wdt:P17 ?country.
   ?country sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
1


In [8]:
# Get information about country 'United States of America'
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
   wd:Q30 ?p ?o.
   ?p sc:name ?pname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('pname', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('pname', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('pname', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1081'), ('pname', 'Human Development Index')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1125'), ('pname', 'Gini coefficient')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pname', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1188'), ('pname', 'Dharma Drum Institute of Liberal Arts place ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1198'), ('pname', 'unemployment rate')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('p

In [10]:
# Get information about country 'United States of America'
queryString = """
SELECT DISTINCT ?of ?name
WHERE {
   wd:Q30 wdt:P31 ?of.
   ?of sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('of', 'http://www.wikidata.org/entity/Q1489259'), ('name', 'superpower')]
[('of', 'http://www.wikidata.org/entity/Q99541706'), ('name', 'historical unrecognized state')]
[('of', 'http://www.wikidata.org/entity/Q1520223'), ('name', 'constitutional republic')]
[('of', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('of', 'http://www.wikidata.org/entity/Q512187'), ('name', 'federal republic')]
[('of', 'http://www.wikidata.org/entity/Q5255892'), ('name', 'democratic republic')]
[('of', 'http://www.wikidata.org/entity/Q6256'), ('name', 'country')]
7


In [12]:
# Get countries
queryString = """
SELECT DISTINCT ?country ?name
WHERE {
   ?country wdt:P31 wd:Q6256.
   ?country sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q16644'), ('name', 'Northern Mariana Islands')]
[('country', 'http://www.wikidata.org/entity/Q207991'), ('name', 'Pahlavi Dynasty')]
[('country', 'http://www.wikidata.org/entity/Q756617'), ('name', 'Danish Realm')]
[('country', 'http://www.wikidata.org/entity/Q786'), ('name', 'Dominican Republic')]
[('country', 'http://www.wikidata.org/entity/Q907112'), ('name', 'Transnistria')]
[('country', 'http://www.wikidata.org/entity/Q244165'), ('name', 'Republic of Artsakh')]
[('country', 'http://www.wikidata.org/entity/Q1005'), ('name', 'The Gambia')]
[('country', 'http://www.wikidata.org/entity/Q1006'), ('name', 'Guinea')]
[('country', 'http://www.wikidata.org/entity/Q1007'), ('name', 'Guinea-Bissau')]
[('country', 'http://www.wikidata.org/entity/Q1008'), ('name', 'Ivory Coast')]
[('country', 'http://www.wikidata.org/entity/Q1009'), ('name', 'Cameroon')]
[('country', 'http://www.wikidata.org/entity/Q1011'), ('name', 'Cape Verde')]
[('country

In [13]:
# Get France
queryString = """
SELECT DISTINCT ?country ?name
WHERE {
   ?country wdt:P31 wd:Q6256.
   ?country sc:name ?name.
   FILTER REGEX(?name, "[Ff]rance")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
1


In [14]:
# Get instances of french 'basketball team' 
queryString = """
SELECT DISTINCT ?bteam ?name
WHERE {
   ?bteam wdt:P31 wd:Q13393265 .
   ?bteam sc:name ?name.
   ?bteam wdt:P17 wd:Q142. #Team in France
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('bteam', 'http://www.wikidata.org/entity/Q795707'), ('name', 'BCM Gravelines')]
[('bteam', 'http://www.wikidata.org/entity/Q1075934'), ('name', 'Cholet Basket')]
[('bteam', 'http://www.wikidata.org/entity/Q2892731'), ('name', 'Paris Université Club')]
[('bteam', 'http://www.wikidata.org/entity/Q404421'), ('name', 'STB Le Havre')]
[('bteam', 'http://www.wikidata.org/entity/Q2887227'), ('name', 'Basket Landes')]
[('bteam', 'http://www.wikidata.org/entity/Q40063'), ('name', 'Le Mans Sarthe Basket')]
[('bteam', 'http://www.wikidata.org/entity/Q2868091'), ('name', 'AS Monaco Basket')]
[('bteam', 'http://www.wikidata.org/entity/Q2352349'), ('name', 'JA Vichy')]
[('bteam', 'http://www.wikidata.org/entity/Q3178083'), ('name', 'JSA Bordeaux Basket')]
[('bteam', 'http://www.wikidata.org/entity/Q3532914'), ('name', 'Toulouse Métropole Basket')]
[('bteam', 'http://www.wikidata.org/entity/Q3549999'), ('name', 'LDLC ASVEL féminin')]
[('bteam', 'http://www.wikidata.org/entity/Q1076356'), ('

In [15]:
# Get properties of 'AS Monaco Basket'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    wd:Q2868091 ?p ?o.
    ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2397'), ('name', 'YouTube channel ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('name', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P646')

In [16]:
# Get properties of 'AS Monaco Basket'
queryString = """
SELECT DISTINCT ?p ?name ?o
WHERE {
    wd:Q2868091 ?p ?o.
    ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue'), ('o', 'http://www.wikidata.org/entity/Q22936222')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league'), ('o', 'http://www.wikidata.org/entity/Q15062840')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location'), ('o', 'http://www.wikidata.org/entity/Q55098')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('o', 'http://www.wikidata.org/entity/Q142')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username'), ('o', 'asmonaco_basket')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username'), ('o', 'asmonaco_basket')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID'), ('o', 'MonacoBasket')]
[('p', 'http://www.wikidata.org/prop/direct/P2397'), ('name', 'YouTube channel ID'), ('o', 'UCYCr10h1MaYZkjfLlGNpbgg')]
[('p', 'http://www.wikidata.org/prop/direct/P31'

In [18]:
# Get instances of french 'basketball team' that started before 1930
queryString = """
SELECT DISTINCT ?bteam ?name
WHERE {
   ?bteam wdt:P31 wd:Q13393265 .
   ?bteam sc:name ?name.
   ?bteam wdt:P17 wd:Q142. #Team in France
   ?bteam wdt:P571 ?date.
   FILTER(?date < "1930-01-01T00:00:00Z"^^xsd:dateTime)
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('bteam', 'http://www.wikidata.org/entity/Q2892731'), ('name', 'Paris Université Club')]
[('bteam', 'http://www.wikidata.org/entity/Q404421'), ('name', 'STB Le Havre')]
[('bteam', 'http://www.wikidata.org/entity/Q2868091'), ('name', 'AS Monaco Basket')]
[('bteam', 'http://www.wikidata.org/entity/Q1630650'), ('name', 'JL Bourg-en-Bresse')]
[('bteam', 'http://www.wikidata.org/entity/Q2868120'), ('name', 'Union Tours Basket Métropole')]
[('bteam', 'http://www.wikidata.org/entity/Q2931972'), ('name', 'Basket CRO Lyon')]
[('bteam', 'http://www.wikidata.org/entity/Q2887216'), ('name', 'BC Orchies')]
[('bteam', 'http://www.wikidata.org/entity/Q3592481'), ('name', 'Étoile de Charleville-Mézières')]
[('bteam', 'http://www.wikidata.org/entity/Q638938'), ('name', 'Paris Basket Racing')]
[('bteam', 'http://www.wikidata.org/entity/Q1682230'), ('name', 'JDA Dijon Basket')]
[('bteam', 'http://www.wikidata.org/entity/Q1419876'), ('name', 'Limoges CSP')]
[('bteam', 'http://www.wikidata.org/ent

In [19]:
queryString = """
SELECT DISTINCT ?bteam ?name
WHERE {
   ?bteam wdt:P31 wd:Q13393265 .
   ?bteam sc:name ?name.
   ?bteam wdt:P17 wd:Q142. #Team in France
   ?bteam wdt:P571 ?date.
   FILTER(?date < "1930-01-01T00:00:00Z"^^xsd:dateTime)
}
"""

print("Results")
x=run_query(queryString)

Results
[('bteam', 'http://www.wikidata.org/entity/Q2892731'), ('name', 'Paris Université Club')]
[('bteam', 'http://www.wikidata.org/entity/Q404421'), ('name', 'STB Le Havre')]
[('bteam', 'http://www.wikidata.org/entity/Q2868091'), ('name', 'AS Monaco Basket')]
[('bteam', 'http://www.wikidata.org/entity/Q1630650'), ('name', 'JL Bourg-en-Bresse')]
[('bteam', 'http://www.wikidata.org/entity/Q2868120'), ('name', 'Union Tours Basket Métropole')]
[('bteam', 'http://www.wikidata.org/entity/Q2931972'), ('name', 'Basket CRO Lyon')]
[('bteam', 'http://www.wikidata.org/entity/Q2887216'), ('name', 'BC Orchies')]
[('bteam', 'http://www.wikidata.org/entity/Q3592481'), ('name', 'Étoile de Charleville-Mézières')]
[('bteam', 'http://www.wikidata.org/entity/Q638938'), ('name', 'Paris Basket Racing')]
[('bteam', 'http://www.wikidata.org/entity/Q1682230'), ('name', 'JDA Dijon Basket')]
[('bteam', 'http://www.wikidata.org/entity/Q1419876'), ('name', 'Limoges CSP')]
[('bteam', 'http://www.wikidata.org/ent

Final query for this task

In [2]:
queryString = """
SELECT DISTINCT ?bteam ?name (year(?date) as ?inception)
WHERE {
   ?bteam wdt:P31 wd:Q13393265 .
   ?bteam sc:name ?name.
   ?bteam wdt:P17 wd:Q142. #Team in France
   ?bteam wdt:P571 ?date.
   FILTER(?date < "1930-01-01T00:00:00Z"^^xsd:dateTime)
}
"""

print("Results")
x=run_query(queryString)

Results
[('bteam', 'http://www.wikidata.org/entity/Q2892731'), ('name', 'Paris Université Club'), ('inception', '1906')]
[('bteam', 'http://www.wikidata.org/entity/Q404421'), ('name', 'STB Le Havre'), ('inception', '1924')]
[('bteam', 'http://www.wikidata.org/entity/Q2868091'), ('name', 'AS Monaco Basket'), ('inception', '1924')]
[('bteam', 'http://www.wikidata.org/entity/Q1630650'), ('name', 'JL Bourg-en-Bresse'), ('inception', '1910')]
[('bteam', 'http://www.wikidata.org/entity/Q2868120'), ('name', 'Union Tours Basket Métropole'), ('inception', '1925')]
[('bteam', 'http://www.wikidata.org/entity/Q2931972'), ('name', 'Basket CRO Lyon'), ('inception', '1927')]
[('bteam', 'http://www.wikidata.org/entity/Q2887216'), ('name', 'BC Orchies'), ('inception', '1922')]
[('bteam', 'http://www.wikidata.org/entity/Q3592481'), ('name', 'Étoile de Charleville-Mézières'), ('inception', '1921')]
[('bteam', 'http://www.wikidata.org/entity/Q638938'), ('name', 'Paris Basket Racing'), ('inception', '1922'

## Task 5
Return all the NBA season (the result set must be a list of couples NBA season IRI, label)

In [20]:
# Get instances of american 'basketball team' 
queryString = """
SELECT DISTINCT ?bteam ?name
WHERE {
   ?bteam wdt:P31 wd:Q13393265 .
   ?bteam sc:name ?name.
   ?bteam wdt:P17 wd:Q30.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('bteam', 'http://www.wikidata.org/entity/Q5144784'), ('name', "Colgate Raiders men's basketball")]
[('bteam', 'http://www.wikidata.org/entity/Q7051115'), ('name', "Norfolk State Spartans men's basketball")]
[('bteam', 'http://www.wikidata.org/entity/Q7397248'), ('name', "Sacred Heart Pioneers men's basketball")]
[('bteam', 'http://www.wikidata.org/entity/Q107876527'), ('name', "Iowa Western Reivers men's basketball")]
[('bteam', 'http://www.wikidata.org/entity/Q15927936'), ('name', 'Westchester Knicks')]
[('bteam', 'http://www.wikidata.org/entity/Q100320348'), ('name', "East Central Tigers men's basketball")]
[('bteam', 'http://www.wikidata.org/entity/Q16201949'), ('name', "DePaul Blue Demons women's basketball")]
[('bteam', 'http://www.wikidata.org/entity/Q16969750'), ('name', 'Jackson State Tigers basketball')]
[('bteam', 'http://www.wikidata.org/entity/Q16982503'), ('name', 'Grand Rapids Drive')]
[('bteam', 'http://www.wikidata.org/entity/Q4705333'), ('name', "Alabama Stat

In [22]:
# Get properties of 'Kobe Bryant'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    wd:Q25369 ?p ?o.
    ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P128

In [24]:
# Get properties of 'Kobe Bryant'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    wd:Q25369 ?p ?o.
    ?p sc:name ?name.
    FILTER REGEX (?name, "^((?!id|Id|ID).)*$") #filter out properties with id in the name
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('name', 'religion')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.

In [26]:
# Get league of 'Kobe Bryant'
queryString = """
SELECT DISTINCT ?league ?name
WHERE {
    wd:Q25369 wdt:P118 ?league.
    ?league sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('league', 'http://www.wikidata.org/entity/Q155223'), ('name', 'National Basketball Association')]
1


In [28]:
# Get properies of 'National Basketball Association'
queryString = """
SELECT DISTINCT ?p ?name ?o ?on
WHERE {
    wd:Q155223 ?p ?o.
    ?p sc:name ?name.
    ?o sc:name ?on.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18536323'), ('on', 'basketball league')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q15986354'), ('on', 'basketball association')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('o', 'http://www.wikidata.org/entity/Q238240'), ('on', 'Eastern Conference')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('name', 'owner of'), ('o', 'http://www.wikidata.org/entity/Q2455323'), ('on', 'NBA TV')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces'), ('o', 'http://www.wikidata.org/entity/Q810343'), ('on', 'Basketball Association of America')]
[('p', 'http://www.wikidata.org/prop/direct/P6118'), ('name', 'season ends'), ('o', 'http://www.wikidata.org/entity/Q118'), ('on', 'April')]
[('p', 'http://www.wikidata.org/prop/direct/P4794'), ('name', 'season

In [37]:
# Get relationships of 'National Basketball Association'
queryString = """
SELECT DISTINCT ?s ?sn ?p ?on
WHERE {
    ?s ?p wd:Q155223.
    ?s sc:name ?sn.
    ?p sc:name ?on.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q7352141'), ('sn', 'Robertson v. National Basketball Association'), ('p', 'http://www.wikidata.org/prop/direct/P1591'), ('on', 'defendant')]
[('s', 'http://www.wikidata.org/entity/Q10950388'), ('sn', 'Template:NBA seasons'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('on', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q106574439'), ('sn', 'Matthew Maurer'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('on', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q22005771'), ('sn', 'Bill Kennedy'), ('p', 'http://www.wikidata.org/prop/direct/P108'), ('on', 'employer')]
[('s', 'http://www.wikidata.org/entity/Q2566198'), ('sn', 'Leon Wood'), ('p', 'http://www.wikidata.org/prop/direct/P108'), ('on', 'employer')]
[('s', 'http://www.wikidata.org/entity/Q3638842'), ('sn', 'Bernie Fryer'), ('p', 'http://www.wikidata.org/prop/direct/P108'), ('on', 'employer')]
[('s', 'http://www.wikidata.org/entity/Q366607'), ('sn'

In [40]:
# Get instance of 'National Basketball Association'
queryString = """
SELECT DISTINCT ?of ?name
WHERE {
    wd:Q155223 wdt:P31 ?of.
    ?of sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('of', 'http://www.wikidata.org/entity/Q18536323'), ('name', 'basketball league')]
[('of', 'http://www.wikidata.org/entity/Q15986354'), ('name', 'basketball association')]
[('of', 'http://www.wikidata.org/entity/Q15991290'), ('name', 'professional sports league')]
[('of', 'http://www.wikidata.org/entity/Q7094076'), ('name', 'online database')]
4


In [41]:
# Get instance of 'National Basketball Association'
queryString = """
SELECT DISTINCT ?of ?name ?sc ?scname
WHERE {
    wd:Q155223 wdt:P31 ?of.
    ?of sc:name ?name.
    ?of wdt:P279 ?sc.
    ?of sc:name ?scname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('of', 'http://www.wikidata.org/entity/Q18536323'), ('name', 'basketball league'), ('sc', 'http://www.wikidata.org/entity/Q623109'), ('scname', 'basketball league')]
[('of', 'http://www.wikidata.org/entity/Q15986354'), ('name', 'basketball association'), ('sc', 'http://www.wikidata.org/entity/Q2485448'), ('scname', 'basketball association')]
[('of', 'http://www.wikidata.org/entity/Q15986354'), ('name', 'basketball association'), ('sc', 'http://www.wikidata.org/entity/Q48204'), ('scname', 'basketball association')]
[('of', 'http://www.wikidata.org/entity/Q15991290'), ('name', 'professional sports league'), ('sc', 'http://www.wikidata.org/entity/Q623109'), ('scname', 'professional sports league')]
[('of', 'http://www.wikidata.org/entity/Q15991290'), ('name', 'professional sports league'), ('sc', 'http://www.wikidata.org/entity/Q1369650'), ('scname', 'professional sports league')]
[('of', 'http://www.wikidata.org/entity/Q15991290'), ('name', 'professional sports league'), ('sc', 

In [45]:
# Get properties of 'Basketball league'
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
    wd:Q18536323 ?p ?o.
    ?p sc:name ?pname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('pname', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category")]
5


In [47]:
# Get super class of 'Basketball league'
queryString = """
SELECT DISTINCT ?super ?name
WHERE {
    wd:Q18536323 wdt:P279 ?super.
    ?super sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('super', 'http://www.wikidata.org/entity/Q623109'), ('name', 'sports league')]
1


In [51]:
# Get properties of 'sports league'
queryString = """
SELECT DISTINCT ?p ?name ?o ?oname
WHERE {
    wd:Q623109 ?p ?o.
    ?p sc:name ?name.
    OPTIONAL { ?o sc:name ?oname }
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P2499'), ('oname', 'league level above')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P2499'), ('oname', 'league level above')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P2500'), ('oname', 'league level below')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P2500'), ('oname', 'league level below')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P3983'), ('oname', 'sports league level')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q10983958'), ('oname', 'Templat

In [52]:
# Get properties of 'sports seasons'
queryString = """
SELECT DISTINCT ?p ?name ?o ?oname
WHERE {
    wd:Q27020041 ?p ?o.
    ?p sc:name ?name.
    OPTIONAL { ?o sc:name ?oname }
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P393'), ('oname', 'edition number')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q51031626'), ('oname', 'sport competition at a multi-sport event')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q16510064'), ('oname', 'sporting event')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q13406554'), ('oname', 'sports competition')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q18608583'), ('oname', 'recurring sporting event')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of'), ('o', 'http://www.wikidata.org/entity/Q18608583'), ('oname', 

In [53]:
# Get instances of 'sports seasons'
queryString = """
SELECT DISTINCT ?ss ?name
WHERE {
    ?ss wdt:P31 wd:Q27020041.
    ?ss sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('ss', 'http://www.wikidata.org/entity/Q1024094'), ('name', "2010 CONCACAF Women's Gold Cup")]
[('ss', 'http://www.wikidata.org/entity/Q10271305'), ('name', '2014 FIFA World Cup venues')]
[('ss', 'http://www.wikidata.org/entity/Q1081152'), ('name', '2010 World Touring Car Championship season')]
[('ss', 'http://www.wikidata.org/entity/Q1082132'), ('name', '2004 UEFA European Under-17 Football Championship')]
[('ss', 'http://www.wikidata.org/entity/Q1122543'), ('name', '1995 UCI Track Cycling World Championships')]
[('ss', 'http://www.wikidata.org/entity/Q1137516'), ('name', '2009–2010 Coupe de France')]
[('ss', 'http://www.wikidata.org/entity/Q1139964'), ('name', '2015 Cricket World Cup')]
[('ss', 'http://www.wikidata.org/entity/Q1142956'), ('name', "2011 Women's Professional Soccer season")]
[('ss', 'http://www.wikidata.org/entity/Q11782976'), ('name', '1994–95 in Ukrainian football')]
[('ss', 'http://www.wikidata.org/entity/Q1191053'), ('name', "2009–10 UEFA Women's Champions

In [58]:
# Explore properties of an instance of 'sport seasons'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    ?ss wdt:P31 wd:Q27020041.
    ?ss ?p ?o.
    ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]


In [60]:
# Explore properties of an instance of 'sport seasons' with sport 'basketball'
queryString = """
SELECT DISTINCT ?ss ?name
WHERE {
    ?ss wdt:P31 wd:Q27020041.
    ?ss wdt:P641 wd:Q5372.        #Sport Basketabll
    ?ss sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('ss', 'http://www.wikidata.org/entity/Q16191059'), ('name', "2013–14 Atlantic Coast Conference men's basketball season")]
[('ss', 'http://www.wikidata.org/entity/Q16192074'), ('name', "2013–14 Southeastern Conference men's basketball season")]
[('ss', 'http://www.wikidata.org/entity/Q19863393'), ('name', "2008–09 Atlantic 10 Conference men's basketball season")]
[('ss', 'http://www.wikidata.org/entity/Q2855716'), ('name', '2012–13 ACB season')]
[('ss', 'http://www.wikidata.org/entity/Q2948730'), ('name', "2010 NCAA Men's Division I Basketball Tournament")]
[('ss', 'http://www.wikidata.org/entity/Q2948731'), ('name', "2012 NCAA Men's Division I Basketball Tournament")]
[('ss', 'http://www.wikidata.org/entity/Q3994982'), ('name', "2013 NCAA Men's Division I Basketball Tournament")]
[('ss', 'http://www.wikidata.org/entity/Q4574588'), ('name', '1971 National Invitation Tournament')]
[('ss', 'http://www.wikidata.org/entity/Q4579916'), ('name', '1981 National Invitation Tournament'

In [62]:
# Explore properties of an instance of 'sport seasons' with sport 'basketball'
queryString = """
SELECT DISTINCT ?ss ?name
WHERE {
    ?ss wdt:P31 wd:Q27020041.
    ?ss wdt:P641 wd:Q5372.        #Sport Basketabll
    ?ss sc:name ?name.
    FILTER REGEX(?name, "[Nn][Bb][Aa]")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('ss', 'http://www.wikidata.org/entity/Q600389'), ('name', '1990–91 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q2264531'), ('name', '1967–68 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q241834'), ('name', '1989–90 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q81513156'), ('name', '2019–20 NBA G League season')]
[('ss', 'http://www.wikidata.org/entity/Q17182411'), ('name', '2013 NBA Development League Draft')]
[('ss', 'http://www.wikidata.org/entity/Q602553'), ('name', '1969–70 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q2294127'), ('name', '1971–72 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q113619'), ('name', '2012–13 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q16191100'), ('name', '2014-15 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q19862716'), ('name', '2015–16 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q93735'), ('name', '1961 NBA Expansion Draft')]
[('ss', 'http://www.wikidata.org/entity/

In [65]:
# Explore properties of an NBA season
queryString = """
SELECT DISTINCT ?p ?name ?o ?oname
WHERE {
    wd:Q600389 ?p ?o.
    ?p sc:name ?name.
    OPTIONAL { ?o sc:name ?oname }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q8172698'), ('oname', 'Category:1990–91 NBA season')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('name', 'time period'), ('o', 'http://www.wikidata.org/entity/Q53389240'), ('oname', '1990-1991 one-year-period')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q27020041'), ('oname', 'sports season')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oname', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q46343203'), ('oname', "men's basketball")]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5372'), ('oname', 'basketball')]
[('p', 'http://www.wikid

In [67]:
# All instances of 'sport seasons' that have as 'sports season of league or competition' the URI of NBA
queryString = """
SELECT DISTINCT ?ss ?name
WHERE {
    ?ss wdt:P31 wd:Q27020041.
    ?ss wdt:P3450 wd:Q155223.
    ?ss sc:name ?name.
}
ORDER BY DESC(?name)
"""

print("Results")
x=run_query(queryString)

Results
[('ss', 'http://www.wikidata.org/entity/Q106495853'), ('name', '2021–22 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q96083208'), ('name', '2020–21 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q63637259'), ('name', '2019–20 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q52702755'), ('name', '2018–19 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q29098111'), ('name', '2017–18 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q30646987'), ('name', '2017 NBA Summer League')]
[('ss', 'http://www.wikidata.org/entity/Q23939796'), ('name', '2016–17 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q24909542'), ('name', '2016 NBA Summer League')]
[('ss', 'http://www.wikidata.org/entity/Q19862716'), ('name', '2015–16 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q16191100'), ('name', '2014-15 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q17001379'), ('name', '2014 NBA Summer League')]
[('ss', 'http://www.wikidata.org/entity/Q111

Final query for this task

In [68]:
queryString = """
SELECT DISTINCT ?ss ?name
WHERE {
    ?ss wdt:P31 wd:Q27020041.
    ?ss wdt:P3450 wd:Q155223.
    ?ss sc:name ?name.
}
ORDER BY DESC(?name)
"""

print("Results")
x=run_query(queryString)

Results
[('ss', 'http://www.wikidata.org/entity/Q106495853'), ('name', '2021–22 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q96083208'), ('name', '2020–21 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q63637259'), ('name', '2019–20 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q52702755'), ('name', '2018–19 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q29098111'), ('name', '2017–18 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q30646987'), ('name', '2017 NBA Summer League')]
[('ss', 'http://www.wikidata.org/entity/Q23939796'), ('name', '2016–17 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q24909542'), ('name', '2016 NBA Summer League')]
[('ss', 'http://www.wikidata.org/entity/Q19862716'), ('name', '2015–16 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q16191100'), ('name', '2014-15 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q17001379'), ('name', '2014 NBA Summer League')]
[('ss', 'http://www.wikidata.org/entity/Q111

## Task 6
Consider only the basketball teams which Michael Jordan and Kobe Bryant played. Return the number of NBA seasons won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team even if they never won a season).

In [71]:
queryString = """
SELECT DISTINCT ?p ?name ?o ?oname
WHERE {
    wd:Q25369 ?p ?o.
    ?p sc:name ?name.
    OPTIONAL{?o sc:name ?oname.}
    FILTER REGEX (?name, "^((?!id|Id|ID).)*$")
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('name', 'educated at'), ('o', 'http://www.wikidata.org/entity/Q6693608'), ('oname', 'Lower Merion High School')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death'), ('o', 'http://www.wikidata.org/entity/Q15729048'), ('oname', 'pending investigation')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial'), ('o', 'http://www.wikidata.org/entity/Q78976032'), ('oname', 'Pacific View Memorial Park')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received'), ('o', 'http://www.wikidata.org/entity/Q1324407'), ('oname', 'Academy Award for Best Live Action Short Film')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of'), ('o', 'http://www.wikidata.org/entity/Q7489331'), ('oname', 'Shaq–Kobe feud')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('name', 'given name'), ('o', 'http://www.wikidata.org/entity/Q21055584'), ('oname', 'Kobe'

In [70]:
# Get the team in which Kobe played
queryString = """
SELECT DISTINCT ?team ?name
WHERE {
    wd:Q25369 wdt:P54 ?team.
    ?team sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers')]
1


In [72]:
# Get the team in which Jordan played
queryString = """
SELECT DISTINCT ?team ?name
WHERE {
    wd:Q41421 wdt:P54 ?team.
    ?team sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q128109'), ('name', 'Chicago Bulls')]
[('team', 'http://www.wikidata.org/entity/Q169165'), ('name', 'Washington Wizards')]
[('team', 'http://www.wikidata.org/entity/Q2747877'), ('name', 'Birmingham Barons')]
[('team', 'http://www.wikidata.org/entity/Q3476102'), ('name', 'Scottsdale Scorpions')]
[('team', 'http://www.wikidata.org/entity/Q4412061'), ('name', "North Carolina Tar Heels men's basketball")]
5


In [74]:
# Get properties of 'Lakers'
queryString = """
SELECT DISTINCT ?p ?name ?o ?oname
WHERE {
    wd:Q121783 ?p ?o.
    ?p sc:name ?name.
    OPTIONAL{?o sc:name ?oname.}
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oname', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5372'), ('oname', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location'), ('o', 'http://www.wikidata.org/entity/Q65'), ('oname', 'Los Angeles')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league'), ('o', 'http://www.wikidata.org/entity/Q155223'), ('oname', 'National Basketball Association')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('name', 'owner of'), ('o', 'http://www.wikidata.org/entity/Q1781055'), ('oname', 'South Bay Lakers')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue'), ('o', 'http://www.wikidata.org/entity/Q186080'), ('oname', 'Staples Center')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name'

In [75]:
# Get all winners of all seasons of NBA
queryString = """
SELECT DISTINCT ?ss ?name ?wonByTeam ?teamName
WHERE {
    ?ss wdt:P31 wd:Q27020041.
    ?ss wdt:P3450 wd:Q155223.
    ?ss sc:name ?name.
    ?ss wdt:P1346 ?wonByTeam.
    ?wonByTeam sc:name ?teamName.
}
ORDER BY DESC(?name)
"""

print("Results")
x=run_query(queryString)

Results
[('ss', 'http://www.wikidata.org/entity/Q96083208'), ('name', '2020–21 NBA season'), ('wonByTeam', 'http://www.wikidata.org/entity/Q169637'), ('teamName', 'Milwaukee Bucks')]
[('ss', 'http://www.wikidata.org/entity/Q63637259'), ('name', '2019–20 NBA season'), ('wonByTeam', 'http://www.wikidata.org/entity/Q121783'), ('teamName', 'Los Angeles Lakers')]
[('ss', 'http://www.wikidata.org/entity/Q52702755'), ('name', '2018–19 NBA season'), ('wonByTeam', 'http://www.wikidata.org/entity/Q132880'), ('teamName', 'Toronto Raptors')]
[('ss', 'http://www.wikidata.org/entity/Q29098111'), ('name', '2017–18 NBA season'), ('wonByTeam', 'http://www.wikidata.org/entity/Q157376'), ('teamName', 'Golden State Warriors')]
[('ss', 'http://www.wikidata.org/entity/Q23939796'), ('name', '2016–17 NBA season'), ('wonByTeam', 'http://www.wikidata.org/entity/Q157376'), ('teamName', 'Golden State Warriors')]
[('ss', 'http://www.wikidata.org/entity/Q19862716'), ('name', '2015–16 NBA season'), ('wonByTeam', 'ht

In [80]:
# Get all winners of all seasons of NBA
queryString = """
SELECT ?wonByTeam ?teamName (COUNT(?ss) as ?victories)
WHERE {
    ?ss wdt:P31 wd:Q27020041.
    ?ss wdt:P3450 wd:Q155223.
    ?ss sc:name ?name.
    ?ss wdt:P1346 ?wonByTeam.
    ?wonByTeam sc:name ?teamName.
}
GROUP BY ?wonByTeam ?teamName
ORDER BY DESC(?victories)
"""

print("Results")
x=run_query(queryString)

Results
[('wonByTeam', 'http://www.wikidata.org/entity/Q121783'), ('teamName', 'Los Angeles Lakers'), ('victories', '17')]
[('wonByTeam', 'http://www.wikidata.org/entity/Q131371'), ('teamName', 'Boston Celtics'), ('victories', '17')]
[('wonByTeam', 'http://www.wikidata.org/entity/Q157376'), ('teamName', 'Golden State Warriors'), ('victories', '6')]
[('wonByTeam', 'http://www.wikidata.org/entity/Q128109'), ('teamName', 'Chicago Bulls'), ('victories', '6')]
[('wonByTeam', 'http://www.wikidata.org/entity/Q159729'), ('teamName', 'San Antonio Spurs'), ('victories', '5')]
[('wonByTeam', 'http://www.wikidata.org/entity/Q138089'), ('teamName', 'Philadelphia 76ers'), ('victories', '3')]
[('wonByTeam', 'http://www.wikidata.org/entity/Q169138'), ('teamName', 'Miami Heat'), ('victories', '3')]
[('wonByTeam', 'http://www.wikidata.org/entity/Q169661'), ('teamName', 'Detroit Pistons'), ('victories', '3')]
[('wonByTeam', 'http://www.wikidata.org/entity/Q161345'), ('teamName', 'Houston Rockets'), ('vic

In [4]:
# Get the team in which Jordan or Bryant played
queryString = """
SELECT DISTINCT ?team ?name
WHERE {
    ?player wdt:P54 ?team.
    ?team sc:name ?name.
    FILTER(?player = wd:Q41421 || ?player = wd:Q25369)
}
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers')]
[('team', 'http://www.wikidata.org/entity/Q128109'), ('name', 'Chicago Bulls')]
[('team', 'http://www.wikidata.org/entity/Q169165'), ('name', 'Washington Wizards')]
[('team', 'http://www.wikidata.org/entity/Q2747877'), ('name', 'Birmingham Barons')]
[('team', 'http://www.wikidata.org/entity/Q3476102'), ('name', 'Scottsdale Scorpions')]
[('team', 'http://www.wikidata.org/entity/Q4412061'), ('name', "North Carolina Tar Heels men's basketball")]
6


In [2]:
# Get the championhips won by Lakers
queryString = """
SELECT ?ss ?ssName
WHERE {
    ?ss wdt:P31 wd:Q27020041.     # BGP for Sport Season
    ?ss wdt:P3450 wd:Q155223.     # BGP for NBA
    wd:Q121783 ^wdt:P1346 ?ss.  # Get season won by the given team
    ?ss sc:name ?ssName.
}
ORDER BY DESC(?name)
"""

print("Results")
x=run_query(queryString)

Results
[('ss', 'http://www.wikidata.org/entity/Q2294127'), ('ssName', '1971–72 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q631240'), ('ssName', '1979–80 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q281094'), ('ssName', '1949–50 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q1132622'), ('ssName', '2001–02 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q1132641'), ('ssName', '1999–2000 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q1132667'), ('ssName', '2000–01 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q1999389'), ('ssName', '1952–53 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q63637259'), ('ssName', '2019–20 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q1130071'), ('ssName', '2009–10 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q1130082'), ('ssName', '2008–09 NBA season')]
[('ss', 'http://www.wikidata.org/entity/Q795353'), ('ssName', '1948–49 BAA season')]
[('ss', 'http://www.wikidata.org/entity/Q20698

In [3]:
# Get the championhips won by Lakers
queryString = """
SELECT COUNT(DISTINCT(?ss)) as ?championships
WHERE {
    ?ss wdt:P31 wd:Q27020041.     # BGP for Sport Season
    ?ss wdt:P3450 wd:Q155223.     # BGP for NBA
    wd:Q121783 ^wdt:P1346 ?ss.    # Get season won by the given team
} 
"""

print("Results")
x=run_query(queryString)

Results
[('championships', '17')]
1


In [4]:
# Get the championhips won by Birmingham Barons
queryString = """
SELECT COUNT(DISTINCT(?ss)) as ?championships
WHERE {
    ?ss wdt:P31 wd:Q27020041.     # BGP for Sport Season
    ?ss wdt:P3450 wd:Q155223.     # BGP for NBA
    wd:Q2747877 ^wdt:P1346 ?ss.    # Get season won by the given team
} 
"""

print("Results")
x=run_query(queryString)

Results
[('championships', '0')]
1


Final query for this task

In [11]:
# Get the number of NBA championships won by all the teams in which Jordan or Bryant played
queryString = """
SELECT DISTINCT ?team ?name COUNT(DISTINCT(?ss)) as ?championships
WHERE {
    ?player wdt:P54 ?team.
    ?team sc:name ?name.
    FILTER(?player = wd:Q41421 || ?player = wd:Q25369)
    
    OPTIONAL {
        ?ss wdt:P31 wd:Q27020041.     # BGP for Sport Season
        ?ss wdt:P3450 wd:Q155223.     # BGP for NBA
        ?team ^wdt:P1346 ?ss.         # Get season won by the given team
    }
    
}
GROUP BY ?team ?name
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q169165'), ('name', 'Washington Wizards'), ('championships', '1')]
[('team', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers'), ('championships', '17')]
[('team', 'http://www.wikidata.org/entity/Q4412061'), ('name', "North Carolina Tar Heels men's basketball"), ('championships', '0')]
[('team', 'http://www.wikidata.org/entity/Q2747877'), ('name', 'Birmingham Barons'), ('championships', '0')]
[('team', 'http://www.wikidata.org/entity/Q3476102'), ('name', 'Scottsdale Scorpions'), ('championships', '0')]
[('team', 'http://www.wikidata.org/entity/Q128109'), ('name', 'Chicago Bulls'), ('championships', '6')]
6


## Task 7
Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, label and #players)

In [70]:
# Get the team in which Kobe played
queryString = """
SELECT DISTINCT ?team ?name
WHERE {
    wd:Q25369 wdt:P54 ?team.
    ?team sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers')]
1


In [5]:
# Get info about relationships of 'Los Angeles Lakers'
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
    wd:Q121783 ?p ?o.
    ?p sc:name ?pname.
    ?o sc:name ?oname.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oname', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5372'), ('oname', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pname', 'headquarters location'), ('o', 'http://www.wikidata.org/entity/Q65'), ('oname', 'Los Angeles')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('pname', 'league'), ('o', 'http://www.wikidata.org/entity/Q155223'), ('oname', 'National Basketball Association')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('pname', 'owner of'), ('o', 'http://www.wikidata.org/entity/Q1781055'), ('oname', 'South Bay Lakers')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('pname', 'home venue'), ('o', 'http://www.wikidata.org/entity/Q186080'), ('oname', 'Staples Center')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), (

In [8]:
# Get all teams that competed in the NBA league
queryString = """
SELECT DISTINCT ?team ?teamName
WHERE {
    ?team wdt:P31 wd:Q13393265. #BGP for Basketball team
    ?team wdt:P118 wd:Q155223.  #BGP to specify the league NBA
    ?team sc:name ?teamName.
}
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q131364'), ('teamName', 'New York Knicks')]
[('team', 'http://www.wikidata.org/entity/Q162954'), ('teamName', 'Denver Nuggets')]
[('team', 'http://www.wikidata.org/entity/Q167253'), ('teamName', 'Portland Trail Blazers')]
[('team', 'http://www.wikidata.org/entity/Q180950'), ('teamName', 'Oklahoma City Thunder')]
[('team', 'http://www.wikidata.org/entity/Q976396'), ('teamName', 'Los Angeles Clippers')]
[('team', 'http://www.wikidata.org/entity/Q166105'), ('teamName', 'Sacramento Kings')]
[('team', 'http://www.wikidata.org/entity/Q170649'), ('teamName', 'Utah Jazz')]
[('team', 'http://www.wikidata.org/entity/Q138089'), ('teamName', 'Philadelphia 76ers')]
[('team', 'http://www.wikidata.org/entity/Q161345'), ('teamName', 'Houston Rockets')]
[('team', 'http://www.wikidata.org/entity/Q164177'), ('teamName', 'Phoenix Suns')]
[('team', 'http://www.wikidata.org/entity/Q172339'), ('teamName', 'New Orleans Pelicans')]
[('team', 'http://www.wikidat

In [10]:
# Get all players of 'Los Angeles Lakers'
queryString = """
SELECT DISTINCT ?player ?name
WHERE {
    wd:Q121783 ^wdt:P54 ?player.
    ?player sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q538009'), ('name', 'Byron Scott')]
[('player', 'http://www.wikidata.org/entity/Q553063'), ('name', 'Steve Blake')]
[('player', 'http://www.wikidata.org/entity/Q2567162'), ('name', 'Jay Vincent')]
[('player', 'http://www.wikidata.org/entity/Q5255286'), ('name', 'Demetrius Calip')]
[('player', 'http://www.wikidata.org/entity/Q2275193'), ('name', 'Mel Counts')]
[('player', 'http://www.wikidata.org/entity/Q2552896'), ('name', 'Coby Karl')]
[('player', 'http://www.wikidata.org/entity/Q2562137'), ('name', 'Joe Kleine')]
[('player', 'http://www.wikidata.org/entity/Q2565049'), ('name', 'Zelmo Beaty')]
[('player', 'http://www.wikidata.org/entity/Q2565541'), ('name', 'Elmore Smith')]
[('player', 'http://www.wikidata.org/entity/Q703908'), ('name', 'Nick Young')]
10


In [11]:
# Get info about ehnic group of 'Kobe Bryant'
queryString = """
SELECT DISTINCT ?eg ?name
WHERE {
    wd:Q25369 wdt:P172 ?eg.
    ?eg sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('eg', 'http://www.wikidata.org/entity/Q49085'), ('name', 'African Americans')]
1


In [12]:
# Get info about ehnic group 'African Americans'
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
    wd:Q49085 ?p ?o.
    ?p sc:name ?pname.
    ?o sc:name ?oname.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q16242628'), ('oname', 'African diaspora in the Americas')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from'), ('o', 'http://www.wikidata.org/entity/Q49297'), ('oname', 'indigenous peoples of the United States')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from'), ('o', 'http://www.wikidata.org/entity/Q49078'), ('oname', 'White Americans')]
[('p', 'http://www.wikidata.org/prop/direct/P3833'), ('pname', 'diaspora'), ('o', 'http://www.wikidata.org/entity/Q385967'), ('oname', 'African diaspora')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q817393'), ('oname', 'Black people')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q28790362'), ('oname', 'ethnic community')]
[(

In [13]:
# Get info about 'country of citizenship' of 'Kobe Bryant'
queryString = """
SELECT DISTINCT ?cc ?name
WHERE {
    wd:Q25369 wdt:P27 ?cc.
    ?cc sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('cc', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
1


In [15]:
# Get info about 'United States of America'
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
    wd:Q30 ?p ?o.
    ?p sc:name ?pname.
    ?o sc:name ?oname.
    FILTER REGEX(?pname, "[Cc]ontinent")
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent'), ('o', 'http://www.wikidata.org/entity/Q49'), ('oname', 'North America')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent'), ('o', 'http://www.wikidata.org/entity/Q538'), ('oname', 'Insular Oceania')]
2


In [19]:
# Get info about 'North America'
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
    wd:Q49 ?p ?o.
    ?p sc:name ?pname.
    ?o sc:name ?oname.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('pname', 'history of topic'), ('o', 'http://www.wikidata.org/entity/Q149527'), ('oname', 'history of North America')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from'), ('o', 'http://www.wikidata.org/entity/Q19464773'), ('oname', 'Northern America and Mexico')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q19464773'), ('oname', 'Northern America and Mexico')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q16'), ('oname', 'Canada')]
[('p', 'http://www.wikidata.org/prop/direct/P47'), ('pname', 'shares border with'), ('o', 'http://www.wikidata.org/entity/Q18'), ('oname', 'South America')]
[('p', 'http://www.wikidata.org/prop/direct/P421'), ('pname', 'located in time zone'), ('o', 'http://www.wikidata.org/entity/Q2085376'), ('oname', 'Atlantic Time Zone')]
[('p', 'h

In [20]:
# Get info about instances of 'Continent'
queryString = """
SELECT DISTINCT ?s ?sname
WHERE {
    ?s wdt:P31 wd:Q5107.
    ?s sc:name ?sname.
}
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q15'), ('sname', 'Africa')]
[('s', 'http://www.wikidata.org/entity/Q18'), ('sname', 'South America')]
[('s', 'http://www.wikidata.org/entity/Q46'), ('sname', 'Europe')]
[('s', 'http://www.wikidata.org/entity/Q48'), ('sname', 'Asia')]
[('s', 'http://www.wikidata.org/entity/Q49'), ('sname', 'North America')]
[('s', 'http://www.wikidata.org/entity/Q51'), ('sname', 'Antarctica')]
[('s', 'http://www.wikidata.org/entity/Q538'), ('sname', 'Insular Oceania')]
[('s', 'http://www.wikidata.org/entity/Q828'), ('sname', 'Americas')]
[('s', 'http://www.wikidata.org/entity/Q3960'), ('sname', 'Australia')]
[('s', 'http://www.wikidata.org/entity/Q2872203'), ('sname', 'mainland Australia')]
[('s', 'http://www.wikidata.org/entity/Q27527'), ('sname', 'Afro-Eurasia')]
[('s', 'http://www.wikidata.org/entity/Q7856408'), ('sname', 'Turtle Island')]
[('s', 'http://www.wikidata.org/entity/Q150408'), ('sname', 'Zealandia')]
13


In [21]:
# Get info about 'South America'
queryString = """
SELECT DISTINCT ?class ?name
WHERE {
    wd:Q18 wdt:P31 ?class.
    ?class sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q5107'), ('name', 'continent')]
[('class', 'http://www.wikidata.org/entity/Q855697'), ('name', 'subcontinent')]
2


In [25]:
# Get info about instances of 'Continent'
queryString = """
SELECT DISTINCT ?s ?sname
WHERE {
    ?s wdt:P31 wd:Q5107.
    ?s sc:name ?sname.
    FILTER NOT EXISTS { ?s wdt:P31 wd:Q855697 }
}
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q15'), ('sname', 'Africa')]
[('s', 'http://www.wikidata.org/entity/Q46'), ('sname', 'Europe')]
[('s', 'http://www.wikidata.org/entity/Q48'), ('sname', 'Asia')]
[('s', 'http://www.wikidata.org/entity/Q51'), ('sname', 'Antarctica')]
[('s', 'http://www.wikidata.org/entity/Q538'), ('sname', 'Insular Oceania')]
[('s', 'http://www.wikidata.org/entity/Q828'), ('sname', 'Americas')]
[('s', 'http://www.wikidata.org/entity/Q3960'), ('sname', 'Australia')]
[('s', 'http://www.wikidata.org/entity/Q2872203'), ('sname', 'mainland Australia')]
[('s', 'http://www.wikidata.org/entity/Q27527'), ('sname', 'Afro-Eurasia')]
[('s', 'http://www.wikidata.org/entity/Q7856408'), ('sname', 'Turtle Island')]
[('s', 'http://www.wikidata.org/entity/Q150408'), ('sname', 'Zealandia')]
11


In [27]:
# Get info about 'Afro-Eurasia'
queryString = """
SELECT DISTINCT ?class ?name
WHERE {
    wd:Q27527 wdt:P31 ?class.
    ?class sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q5107'), ('name', 'continent')]
[('class', 'http://www.wikidata.org/entity/Q55833'), ('name', 'supercontinent')]
2


In [28]:
# Get info about instances of 'Continent'
queryString = """
SELECT DISTINCT ?s ?sname
WHERE {
    ?s wdt:P31 wd:Q5107.
    ?s sc:name ?sname.
    FILTER NOT EXISTS { ?s wdt:P31 wd:Q855697 } #filter out sub continents
    FILTER NOT EXISTS { ?s wdt:P31 wd:Q55833 } #filter out supercontinent
}
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q15'), ('sname', 'Africa')]
[('s', 'http://www.wikidata.org/entity/Q46'), ('sname', 'Europe')]
[('s', 'http://www.wikidata.org/entity/Q48'), ('sname', 'Asia')]
[('s', 'http://www.wikidata.org/entity/Q51'), ('sname', 'Antarctica')]
[('s', 'http://www.wikidata.org/entity/Q538'), ('sname', 'Insular Oceania')]
[('s', 'http://www.wikidata.org/entity/Q3960'), ('sname', 'Australia')]
[('s', 'http://www.wikidata.org/entity/Q2872203'), ('sname', 'mainland Australia')]
[('s', 'http://www.wikidata.org/entity/Q7856408'), ('sname', 'Turtle Island')]
[('s', 'http://www.wikidata.org/entity/Q150408'), ('sname', 'Zealandia')]
9


In [29]:
# Get info about 'mainland Australia'
queryString = """
SELECT DISTINCT ?class ?name
WHERE {
    wd:Q2872203 wdt:P31 ?class.
    ?class sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q205895'), ('name', 'landmass')]
[('class', 'http://www.wikidata.org/entity/Q5107'), ('name', 'continent')]
2


In [30]:
# Get info about 'Australia'
queryString = """
SELECT DISTINCT ?class ?name
WHERE {
    wd:Q3960 wdt:P31 ?class.
    ?class sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q5107'), ('name', 'continent')]
1


In [31]:
# Get all players of 'Los Angeles Lakers' with their continent
queryString = """
SELECT DISTINCT ?player ?name ?continent ?continentName
WHERE {
    ?team ^wdt:P54 ?player.
    ?player sc:name ?name.
    ?player wdt:P27 ?c.        #BGP for 'country of citizenship'
    ?c wdt:P30 ?continent.     #BGP for 'continent' for country
    ?continent sc:name ?continentName.
    
    FILTER(?team = wd:Q121783) #filter only players of 'Lakers'
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q1374718'), ('name', 'Ime Udoka'), ('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa')]
[('player', 'http://www.wikidata.org/entity/Q2329803'), ('name', 'Soumaila Samake'), ('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa')]
[('player', 'http://www.wikidata.org/entity/Q381762'), ('name', 'D. J. Mbenga'), ('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa')]
[('player', 'http://www.wikidata.org/entity/Q559449'), ('name', 'Christian Eyenga'), ('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa')]
[('player', 'http://www.wikidata.org/entity/Q716480'), ('name', 'Marcelinho Huertas'), ('continent', 'http://www.wikidata.org/entity/Q18'), ('continentName', 'South America')]
[('player', 'http://www.wikidata.org/entity/Q116551'), ('name', 'Ronny Turiaf'), ('continent', 'http://www.wikidata.org/entity/Q46'), ('continentName', 'Europ

In [36]:
# Get the number of players per continent for the team 'Los Angeles Lakers'
queryString = """
SELECT DISTINCT ?continent ?continentName (COUNT(DISTINCT(?player)) as ?players)
WHERE {
    ?team ^wdt:P54 ?player.
    ?player sc:name ?name.
    ?player wdt:P27 ?c.        #BGP for 'country of citizenship'
    ?c wdt:P30 ?continent.     #BGP for 'continent' for country
    ?continent sc:name ?continentName.
    
    FILTER(?team = wd:Q121783) #filter only players of 'Lakers'
}
GROUP BY ?continent ?continentName
"""

print("Results")
x=run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q46'), ('continentName', 'Europe'), ('players', '25')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('continentName', 'Insular Oceania'), ('players', '399')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa'), ('players', '4')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('continentName', 'Asia'), ('players', '7')]
[('continent', 'http://www.wikidata.org/entity/Q18'), ('continentName', 'South America'), ('players', '1')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('continentName', 'North America'), ('players', '405')]
6


In [37]:
# Get all players of 'Los Angeles Lakers' with their citizenship continent
queryString = """
SELECT DISTINCT ?player ?name ?continent ?continentName
WHERE {
    ?team ^wdt:P54 ?player.
    ?player sc:name ?name.
    ?player wdt:P27 ?c.        #BGP for 'country of citizenship'
    ?c wdt:P30 ?continent.     #BGP for 'continent' for country
    ?continent sc:name ?continentName.
    
    FILTER(?team = wd:Q121783) #filter only players of 'Lakers'
}
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q1374718'), ('name', 'Ime Udoka'), ('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa')]
[('player', 'http://www.wikidata.org/entity/Q2329803'), ('name', 'Soumaila Samake'), ('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa')]
[('player', 'http://www.wikidata.org/entity/Q381762'), ('name', 'D. J. Mbenga'), ('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa')]
[('player', 'http://www.wikidata.org/entity/Q559449'), ('name', 'Christian Eyenga'), ('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa')]
[('player', 'http://www.wikidata.org/entity/Q716480'), ('name', 'Marcelinho Huertas'), ('continent', 'http://www.wikidata.org/entity/Q18'), ('continentName', 'South America')]
[('player', 'http://www.wikidata.org/entity/Q116551'), ('name', 'Ronny Turiaf'), ('continent', 'http://www.wikidata.org/entity/Q46'), ('continentName', 'Europ

In [38]:
# Get the number of players per continent for each NBA team
queryString = """
SELECT DISTINCT ?continent ?continentName (COUNT(DISTINCT(?player)) as ?players)
WHERE {
    ?team ^wdt:P54 ?player.
    ?player sc:name ?name.
    ?player wdt:P27 ?c.        #BGP for 'country of citizenship'
    ?c wdt:P30 ?continent.     #BGP for 'continent' for country
    ?continent sc:name ?continentName.
    
    {
    
        # Get all teams that competed in the NBA league
        SELECT DISTINCT ?team ?teamName
        WHERE {
            ?team wdt:P31 wd:Q13393265. #BGP for Basketball team
            ?team wdt:P118 wd:Q155223.  #BGP to specify the league NBA
            ?team sc:name ?teamName.
        }
        
    }

}
GROUP BY ?continent ?continentName
"""

print("Results")
x=run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q46'), ('continentName', 'Europe'), ('players', '304')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('continentName', 'Insular Oceania'), ('players', '3799')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa'), ('players', '61')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('continentName', 'Asia'), ('players', '73')]
[('continent', 'http://www.wikidata.org/entity/Q5401'), ('continentName', 'Eurasia'), ('players', '17')]
[('continent', 'http://www.wikidata.org/entity/Q18'), ('continentName', 'South America'), ('players', '37')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('continentName', 'North America'), ('players', '3849')]
7


Final query for this task

In [4]:
# Get the number of players per continent for each NBA team
queryString = """
SELECT DISTINCT ?continent ?continentName (COUNT(DISTINCT(?player)) as ?players)
WHERE {
    ?t ^wdt:P54 ?player.
    ?player wdt:P27 ?c.        #BGP for 'country of citizenship'
    ?c wdt:P30 ?continent.     #BGP for 'continent' for country
    ?continent sc:name ?continentName.
    
    FILTER(?t = ?team)
    {
        # Get all teams that competed in the NBA league
        SELECT DISTINCT ?team ?teamName
        WHERE {
            ?team wdt:P31 wd:Q13393265. #BGP for Basketball team
            ?team wdt:P118 wd:Q155223.  #BGP to specify the league NBA
            ?team sc:name ?teamName.
        }   
    }

}
GROUP BY ?continent ?continentName
"""

print("Results")
x=run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q46'), ('continentName', 'Europe'), ('players', '304')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('continentName', 'Insular Oceania'), ('players', '3799')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa'), ('players', '61')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('continentName', 'Asia'), ('players', '73')]
[('continent', 'http://www.wikidata.org/entity/Q5401'), ('continentName', 'Eurasia'), ('players', '17')]
[('continent', 'http://www.wikidata.org/entity/Q18'), ('continentName', 'South America'), ('players', '37')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('continentName', 'North America'), ('players', '3849')]
7


## Task 8
Identify the BGP for the NBA Defensive Player of the Year Award (see https://en.wikipedia.org/wiki/NBA_Defensive_Player_of_the_Year_Award)

In [41]:
# Get awards of 'Kobe Bryant'
queryString = """
SELECT DISTINCT ?award ?name
WHERE {
    wd:Q25369 wdt:P166 ?award.
    ?award sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q1324407'), ('name', 'Academy Award for Best Live Action Short Film')]
[('award', 'http://www.wikidata.org/entity/Q1465181'), ('name', 'NBA All-Defensive Team')]
[('award', 'http://www.wikidata.org/entity/Q222047'), ('name', 'NBA Most Valuable Player Award')]
[('award', 'http://www.wikidata.org/entity/Q31391'), ('name', 'NBA All-Star Game Kobe Bryant Most Valuable Player Award')]
[('award', 'http://www.wikidata.org/entity/Q4835465'), ('name', 'BET Award for Athlete of the Century')]
[('award', 'http://www.wikidata.org/entity/Q674359'), ('name', 'All-NBA Team')]
[('award', 'http://www.wikidata.org/entity/Q739499'), ('name', 'Bill Russell NBA Finals Most Valuable Player Award')]
[('award', 'http://www.wikidata.org/entity/Q943106'), ('name', 'Best NBA Player ESPY Award')]
[('award', 'http://www.wikidata.org/entity/Q7883073'), ('name', 'Under Armour Undeniable Performance ESPY Award')]
9


In [44]:
# Get awards of 'Michael Jordan'
queryString = """
SELECT DISTINCT ?award ?name
WHERE {
    wd:Q41421 wdt:P166 ?award.
    ?award sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q1061233'), ('name', "L'Équipe Champion of Champions")]
[('award', 'http://www.wikidata.org/entity/Q17144'), ('name', 'Presidential Medal of Freedom')]
[('award', 'http://www.wikidata.org/entity/Q743122'), ('name', 'Associated Press Athlete of the Year')]
[('award', 'http://www.wikidata.org/entity/Q137003'), ('name', '50 Greatest Players in NBA History')]
[('award', 'http://www.wikidata.org/entity/Q1418009'), ('name', 'NBA All-Rookie Team')]
[('award', 'http://www.wikidata.org/entity/Q1465181'), ('name', 'NBA All-Defensive Team')]
[('award', 'http://www.wikidata.org/entity/Q1641970'), ('name', 'John R. Wooden Award')]
[('award', 'http://www.wikidata.org/entity/Q1653207'), ('name', 'Sports Illustrated Sportsperson of the Year')]
[('award', 'http://www.wikidata.org/entity/Q222047'), ('name', 'NBA Most Valuable Player Award')]
[('award', 'http://www.wikidata.org/entity/Q2404133'), ('name', 'Oscar Robertson Trophy')]
[('award', 'http://www

Final query for this task

In [45]:
# Get awards of 'Michael Jordan'
queryString = """
SELECT DISTINCT ?award ?name
WHERE {
    wd:Q41421 wdt:P166 ?award.
    ?award sc:name ?name.
    FILTER REGEX(?name, "NBA [Dd]efensive*")
}
"""

print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q845884'), ('name', 'NBA Defensive Player of the Year Award')]
1


## Task 9
Consider the players who won the NBA Defensive Player of the Year Award. Return for these players the number of NBA Defensive Player of the Year Award won in their career (the result set must be triples of player IRI, label and #victories)

In [47]:
# Get who won 'NBA Defensive Player of the Year Award'
queryString = """
SELECT DISTINCT ?player ?name
WHERE {
    ?player wdt:P166 wd:Q845884.
    ?player sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q177928'), ('name', 'Sidney Moncrief')]
[('player', 'http://www.wikidata.org/entity/Q310968'), ('name', 'Alonzo Mourning')]
[('player', 'http://www.wikidata.org/entity/Q971538'), ('name', 'Michael Cooper')]
[('player', 'http://www.wikidata.org/entity/Q189240'), ('name', 'Kevin Garnett')]
[('player', 'http://www.wikidata.org/entity/Q41421'), ('name', 'Michael Jordan')]
[('player', 'http://www.wikidata.org/entity/Q273256'), ('name', 'Hakeem Olajuwon')]
[('player', 'http://www.wikidata.org/entity/Q211876'), ('name', 'David Robinson')]
[('player', 'http://www.wikidata.org/entity/Q361134'), ('name', 'Marcus Camby')]
[('player', 'http://www.wikidata.org/entity/Q295322'), ('name', 'Metta Sandiford-Artest')]
[('player', 'http://www.wikidata.org/entity/Q926248'), ('name', 'Kawhi Leonard')]
[('player', 'http://www.wikidata.org/entity/Q311735'), ('name', 'Joakim Noah')]
[('player', 'http://www.wikidata.org/entity/Q449245'), ('name', 'Alvin Rober

In [48]:
# Get properties of 'NBA Defensive Player of the Year Award'
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
    wd:Q845884 ?p ?o .
    ?p sc:name ?pname.
    ?o sc:name ?oname.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('o', 'http://www.wikidata.org/entity/Q177928'), ('oname', 'Sidney Moncrief')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q618779'), ('oname', 'award')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('o', 'http://www.wikidata.org/entity/Q310968'), ('oname', 'Alonzo Mourning')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5372'), ('oname', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('o', 'http://www.wikidata.org/entity/Q971538'), ('oname', 'Michael Cooper')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('o', 'http://www.wikidata.org/entity/Q189240'), ('oname', 'Kevin Garnett')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('o', 'http://www.w

In [49]:
# Edges that connects 'Kevin Garnett' with 'NBA Defensive Player of the Year Award'
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
    wd:Q189240 ?p wd:Q845884 .
    ?p sc:name ?pname.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
1


In [50]:
# Edges that connects 'NBA Defensive Player of the Year Award' with 'Kevin Garnett' 
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
    wd:Q845884 ?p wd:Q189240 .
    ?p sc:name ?pname.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
1


In [52]:
# Awards won by 'Kevin Garnett'
queryString = """
SELECT DISTINCT ?award ?awname
WHERE {
    wd:Q189240 wdt:P166 ?award .
    ?award sc:name ?awname.
}
"""

print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q1501719'), ('awname', 'J. Walter Kennedy Citizenship Award')]
[('award', 'http://www.wikidata.org/entity/Q1465181'), ('awname', 'NBA All-Defensive Team')]
[('award', 'http://www.wikidata.org/entity/Q222047'), ('awname', 'NBA Most Valuable Player Award')]
[('award', 'http://www.wikidata.org/entity/Q31391'), ('awname', 'NBA All-Star Game Kobe Bryant Most Valuable Player Award')]
[('award', 'http://www.wikidata.org/entity/Q674359'), ('awname', 'All-NBA Team')]
[('award', 'http://www.wikidata.org/entity/Q845884'), ('awname', 'NBA Defensive Player of the Year Award')]
[('award', 'http://www.wikidata.org/entity/Q943106'), ('awname', 'Best NBA Player ESPY Award')]
7


In [ ]:
# Get sub properties of P1346 'winner'
queryString = """
SELECT DISTINCT ?sp ?name
WHERE {
    ?sp wdt:P1647 wdt:P1346 .
    ?sp sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [4]:
# Get players that won 'NBA Defensive Player of the Year Award'
queryString = """
SELECT ?player ?name
WHERE {
    wd:Q845884 wdt:P1346 ?player .
    ?player sc:name ?name.
}
ORDER BY (?name)
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q310968'), ('name', 'Alonzo Mourning')]
[('player', 'http://www.wikidata.org/entity/Q449245'), ('name', 'Alvin Robertson')]
[('player', 'http://www.wikidata.org/entity/Q295364'), ('name', 'Ben Wallace')]
[('player', 'http://www.wikidata.org/entity/Q211876'), ('name', 'David Robinson')]
[('player', 'http://www.wikidata.org/entity/Q201608'), ('name', 'Dennis Rodman')]
[('player', 'http://www.wikidata.org/entity/Q313481'), ('name', 'Dikembe Mutombo')]
[('player', 'http://www.wikidata.org/entity/Q25278'), ('name', 'Dwight Howard')]
[('player', 'http://www.wikidata.org/entity/Q309872'), ('name', 'Gary Payton')]
[('player', 'http://www.wikidata.org/entity/Q273256'), ('name', 'Hakeem Olajuwon')]
[('player', 'http://www.wikidata.org/entity/Q311735'), ('name', 'Joakim Noah')]
[('player', 'http://www.wikidata.org/entity/Q926248'), ('name', 'Kawhi Leonard')]
[('player', 'http://www.wikidata.org/entity/Q189240'), ('name', 'Kevin Garnett')]
[('pla

In [11]:
# Edges that connects 'NBA Defensive Player of the Year Award' with 'Sidney Moncrief' 
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
    wd:Q845884 ?p wd:Q177928 .
    ?p sc:name ?pname.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
1


In [12]:
# Get sub properties of 'winner'
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
    wdt:P1346 wdt:P1647 ?sp.
    ?sp sc:name ?pname.
}
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [2]:
# Get super property of 'winner'
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
    ?sp wdt:P1647 wdt:P1346.
    ?sp sc:name ?pname.
}
"""

print("Results")
x=run_query(queryString)

Results
Empty


Final query for this task

I cannot find any information about the year in which a player has won the award. 

By looking at the info in the [URL](https://en.wikipedia.org/wiki/NBA_Defensive_Player_of_the_Year_Award) Sidney Moncrief won multiple times the award but in the database there is only one link that connects the award with Sidney Moncrief. 

I tried to explore if there was any sub property that could lead me to the times or the point in time in which the award has been assigned but without succeeding. Thus the only information I can extract is which player has won at least one time the 'NBA Defensive Player of the Year Award'

In [14]:
# Get players that won 'NBA Defensive Player of the Year Award'
queryString = """
SELECT ?player ?name
WHERE {
    wd:Q845884 wdt:P1346 ?player .
    ?player sc:name ?name.
}
ORDER BY (?name)
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q310968'), ('name', 'Alonzo Mourning')]
[('player', 'http://www.wikidata.org/entity/Q449245'), ('name', 'Alvin Robertson')]
[('player', 'http://www.wikidata.org/entity/Q295364'), ('name', 'Ben Wallace')]
[('player', 'http://www.wikidata.org/entity/Q211876'), ('name', 'David Robinson')]
[('player', 'http://www.wikidata.org/entity/Q201608'), ('name', 'Dennis Rodman')]
[('player', 'http://www.wikidata.org/entity/Q313481'), ('name', 'Dikembe Mutombo')]
[('player', 'http://www.wikidata.org/entity/Q25278'), ('name', 'Dwight Howard')]
[('player', 'http://www.wikidata.org/entity/Q309872'), ('name', 'Gary Payton')]
[('player', 'http://www.wikidata.org/entity/Q273256'), ('name', 'Hakeem Olajuwon')]
[('player', 'http://www.wikidata.org/entity/Q311735'), ('name', 'Joakim Noah')]
[('player', 'http://www.wikidata.org/entity/Q926248'), ('name', 'Kawhi Leonard')]
[('player', 'http://www.wikidata.org/entity/Q189240'), ('name', 'Kevin Garnett')]
[('pla